# S3 - Challenge
* StelllarAlgo Data Science
* Ryan Kazmerik & Nakisa Rad
* June 7, 2022

## What we want to do:
1. Read the value of the SSM parameter
2. Use the values in the SSM parameter to form a connection string to the MSSQL database listed in the parameter
3. Read the contents of the ds.customerScores table into a dataframe

In [1]:
import boto3 
import json
import pandas as pd
import pyodbc

In [2]:
! aws sso login --profile Stellaralgo-DataScienceAdmin

Attempting to automatically open the SSO authorization page in your default browser.
If the browser does not open or you wish to use a different device to authorize this request, open the following URL:

https://device.sso.us-east-1.amazonaws.com/

Then enter the code:

GLTK-NLFB
Successully logged into Start URL: https://stellaralgo.awsapps.com/start#/


### Get a list of the Parameters we need

In [3]:
PROFILE_NAME='Stellaralgo-DataScienceAdmin'

In [4]:
# creating s3 client connection
SESSION = boto3.setup_default_session(profile_name=PROFILE_NAME)
CLIENT = boto3.client('ssm')
SSM_PARAM_NAME = '/customer/model-retention/ai/db-connections/data-sci-retention/database-read'

### Read the value of the SSM parameter

In [5]:
response = CLIENT.get_parameter(Name= SSM_PARAM_NAME, WithDecryption=True)
# we need to conver the result to dict because the actual reslut is string
param_value = json.loads(response['Parameter']['Value'])

### Use the values in the SSM parameter to form a connection string to the MSSQL database listed in the parameter

In [6]:
# connect to SQL Server.
SERVER = '34.206.73.189' 
DATABASE = param_value['database'] 
USERNAME = param_value['username'] 
PASSWORD = param_value['password']
CNXN = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+SERVER+';DATABASE='+DATABASE+';UID='+USERNAME+';PWD='+ PASSWORD)

### Read the contents of the ds.customerScores table into a dataframe

In [9]:
cursor = CNXN.cursor()

storedProc = (
    f"""select top 10 * from ds.customerScores"""
)

df = pd.read_sql(storedProc, CNXN)

CNXN.commit()
cursor.close()

df.head()

,dimcustomermasterid,buyer_score,tenuredays,attendancePercentage,mostRecentAttendance,year,lkupclientid,productGrouping,seasonYear,insertDate
0,349335093,0.9115,2190,0.555556,2022-01-17,2021,33,Full Season,2021,01-22-2022 09:33:41
1,349335290,0.3554,1825,1.037037,2022-01-17,2021,33,Full Season,2021,01-22-2022 09:33:41
2,349335440,0.3491,1825,1.142857,2022-01-17,2021,33,Partial,2021,01-22-2022 09:33:41
3,349335508,0.359,2190,1.037037,2022-01-17,2021,33,Full Season,2021,01-22-2022 09:33:41
4,349344774,0.8677,1460,0.814815,2022-01-17,2021,33,Full Season,2021,01-22-2022 09:33:41
